In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import datetime

# This code is derived from AWS SageMaker Samples:
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_electricity
# https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/deepar_synthetic

<h2>DeepAR - Kaggle Bike Sharing Demand Dataset</h2>
<h4>Prepare Bike Rental Data for DeepAR training</h4>  
<quote>We will store the total rental count, registered rental count, and casual rental count data as a time series.</quote>
<quote>For each of the targets <b>total, registered, casual</b>, following JSON is structured as: Start Time, Array of target values, optional dynamic features and categories</quote>
<quote>Frequency of time series data (for example, hourly, daily, monthly and so forth) is specified using hyperparameter</quote>

<h4>To download original dataset, sign-in and download from this link: https://www.kaggle.com/c/bike-sharing-demand/data</h4>
<br>
None of these features are used: ['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed']<br>
       
Start Time From: ['datetime'] <br>
Target Feature: [<b>'count','registered','casual'</b>]<br>
Frequency: 'Hourly' <br>

Objective: <quote>You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period (Ref: Kaggle.com)</quote>

In [ ]:
target_values = ['count','registered','casual']

# controls if categories (in this case we are using it to indicate one of the above rentals) needs to be
# included in the training and test data
with_categories = False

# Set datetime column as index to work with data based on Date/Time
df = pd.read_csv('train.csv', parse_dates=['datetime'],index_col=0)
df_test = pd.read_csv('test.csv', parse_dates=['datetime'],index_col=0)

In [ ]:
# Minimum time and Maximum Time in Training CSV file
df.index.min(),df.index.max()

In [ ]:
# Minimum time and Maximum Time in test CSV file
df_test.index.min(),df_test.index.max()

In [ ]:
df_test.head(25)

In [ ]:
df['2011-01']['count'].plot()

In [ ]:
# We need to let DeepAR know how far in the future predictions can be made using prediction_length hyperparameter
# Let's look at how many hours we need to predict in a month using test.csv data file
hours_to_predict = []
print ('Check maximum hours we need to predict')
# Group by year,month
predict_window = df_test.groupby([df_test.index.year,df_test.index.month])
for i,x in predict_window:
    delta = x.index.max() - x.index.min() 
    hours = np.ceil(delta.total_seconds()/3600)
    hours_to_predict.append(hours)
    print ("{0}, Hours:{1}".format(i, hours))

print ("Maximum Prediction Length in Hours: ", np.max(hours_to_predict))

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html
    
freq='H' # Timeseries consists Hourly Data and we need to predict hourly rental count

# how far in the future predictions can be made
# 12 days worth of hourly forecast 
prediction_length = 288 

# aws recommends setting context same as prediction length as a starting point. 
# This controls how far in the past the network can see
context_length = 288

In [ ]:
dt_predict_max = pd.Timestamp("2012-12-31 23:00:00", freq=freq) # 2012-12-31 23:00 alt way..pd.datetime(2012,12,31,23,0,0)

dt_dataset_start_time = pd.Timestamp("2011-01-01 00:00:00", freq=freq)
dt_dataset_end_time = pd.Timestamp("2012-12-19 23:00:00", freq=freq)

# use for model training
# Start time is the first row provided by kaggle
# Training TS end time ensures some data is withheld for model testing
# 12 days worth of training data is withheld for testing
dt_train_range = (dt_dataset_start_time,
                  dt_dataset_end_time - datetime.timedelta(hours=12*24) )

# Use entire data for testing
# We can compare predicted values vs actual (i.e. last 12 days is withheld for testing and model hasn't seen that data)
dt_test_range = (dt_dataset_start_time, 
                 dt_dataset_end_time) 

In [ ]:
dt_predict_max,dt_predict_max+1

In [ ]:
# Let's see if there are gaps in timesteps
def is_missing_steps(df,start,end,freq='D'):
    dt_range = pd.date_range(start=start,end=end,freq=freq)
    return not dt_range.equals(df[start:end].index)

def get_missing_steps(df,start,end,freq='D'):
    dt_range = pd.date_range(start=start,end=end,freq=freq)
    return dt_range.difference(df[start:end].index)    

# List timeseries with only NaNs
# They can be removed
def timeseries_with_only_nans(df):
    l = []
    for col in df.columns:
        if pd.isna(df[col].min()):
            #print (col)
            l.append(col)
    return l

In [ ]:
is_missing_steps(df, '2011-01-01 00:00:00', '2011-01-19 23:00:00','H')

In [ ]:
get_missing_steps(df, '2011-01-01 00:00:00', '2011-01-19 23:00:00','H')

In [ ]:
df['2011-01-02 00:00:00':'2011-01-02 14:00:00']

In [ ]:
df['2011-01-02 00:00:00':'2011-01-02 14:00:00']['count'].plot()

In [ ]:
df = df.resample('1h').mean()

In [ ]:
df['2011-01-02 00:00:00':'2011-01-02 14:00:00']

In [ ]:
df['2011-01-02 00:00:00':'2011-01-02 14:00:00']['count'].plot(title='Missing values in training data')

In [ ]:
df['2012-01':'2012-02']['count'].plot()

In [ ]:
target_values

In [ ]:
df[dt_test_range[0]:dt_test_range[1]]['count'].tail()

In [ ]:
dt_test_range

In [ ]:
dt_train_range

In [ ]:
time_series_test = []
time_series_training = []

for target in target_values:
    time_series_test.append(df[dt_test_range[0]:dt_test_range[1]][target])
    time_series_training.append(df[dt_train_range[0]:dt_train_range[1]][target])

In [ ]:
time_series_test[0][:5],time_series_test[1][:5],time_series_test[2][:5]

In [ ]:
time_series_training[0][:5],time_series_training[1][:5],time_series_training[2][:5]

In [ ]:
time_series_test[0].plot(label='test')
time_series_training[0].plot(label='train')#, ls=':')
plt.legend()
plt.show()

In [ ]:
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]  

def encode_dynamic_feat(dynamic_feat):  
    l = []
    for col in dynamic_feat:
        assert (not dynamic_feat[col].isna().any()), col  + ' has NaN'             
        l.append(dynamic_feat[col].tolist())
    return l

def series_to_obj(ts, cat=None, dynamic_feat=None):
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = encode_dynamic_feat(dynamic_feat)
    return obj

def series_to_jsonline(ts, cat=None, dynamic_feat=None):
    return json.dumps(series_to_obj(ts, cat, dynamic_feat))   

In [ ]:
list(time_series_training[0][:5])

In [ ]:
series_to_obj(time_series_training[0][:5],[0] if with_categories else None)

In [ ]:
series_to_jsonline(time_series_training[0][:5],[0] if with_categories else None)

In [ ]:
encoding = "utf-8"
cat_idx = 0

train_file_name = "train.json"
test_file_name = "test.json"

if with_categories:
    train_file_name = "train_with_categories.json"
    test_file_name = "test_with_categories.json"

with open(train_file_name, 'wb') as fp:
    for ts in time_series_training:
        fp.write(series_to_jsonline(ts,[cat_idx] if with_categories else None).encode(encoding))
        fp.write('\n'.encode(encoding))
        cat_idx += 1

In [ ]:
cat_idx = 0
with open(test_file_name, 'wb') as fp:
    for ts in time_series_test:
        fp.write(series_to_jsonline(ts,[cat_idx] if with_categories else None).encode(encoding))
        fp.write('\n'.encode(encoding))
        cat_idx += 1

In [ ]:
df.to_csv('all_data.csv',index=True,index_label='datetime')

In [ ]:
for ts in time_series_test:
    print (len(ts),ts.name)

In [ ]:
for ts in time_series_training:
    print (len(ts),ts.name)